In [ ]:
use schema PYTHON_WORKLOADS.BRIANS;

In [ ]:
CREATE OR REPLACE function get_secret_type()
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION = 3.8
HANDLER = 'get_secret'
EXTERNAL_ACCESS_INTEGRATIONS = (qualtrics_apis_access_integration)
SECRETS = ('cred' = Exit_Qualtrics_apiToken )
AS
$$
import _snowflake
def get_secret():
  secret_type = _snowflake.get_generic_secret_string('cred')
  return secret_type
$$;

In [ ]:
select PYTHON_WORKLOADS.BRIANS.get_secret_type() as APIKEY

In [ ]:
api_key_from_secret=token_from_secret.to_pandas()['APIKEY'].iloc[0]

In [ ]:
print(api_key_from_secret)

In [ ]:
from snowflake.snowpark.context import get_active_session

# Get active Snowpark session
session = get_active_session()

# Verify the current warehouse
current_warehouse_name = session.get_current_warehouse()
print(current_warehouse_name)

In [ ]:
def left(s, amount):
    return s[:amount]

In [ ]:
# import datetime
# import tempfile
# import requests
# import csv
# import pandas as pd
# import _snowflake

# # Define Qualtrics file configurations
# qualtrics_files = [
#     {
#         'automationId': 'AU_wlsKb91R5uyarA8',
#         'file_name': 'Exit_survey',
#         'table': 'bi.sales.exit_survey'
#     }
# ]

# # API token and data center
# # Fetch the secret
# api_secret_query = "select get_secret_type()"
# apiToken = session.sql(api_secret_query).collect()[0][0]
# #apiToken = "token"
# dataCenter = "az1"
# headers = { "x-api-token": apiToken }

In [ ]:
select *, 1 as is_exit from bi.sales.exit_survey limit 2 --where termination_date between current_date-7 and current_date

In [ ]:
import os
#os.chdir('/content/drive/MyDrive/Qualtrics/sendsurveys')

#!pip install snowflake-connector-python

#libraries
import requests
import snowflake.connector
import csv
import sys
import os
import datetime
import tempfile






apiToken=api_key_from_secret

dataCenter = "az1"
print(apiToken)

headers = { "x-api-token": apiToken }

def left(s, amount):
    return s[:amount]

qualtrics_files = [

{
        'automationId': 'AU_cOPoUihOatQkzzh',
        'file_name': 'Exit_survey',
        'table': 'bi.sales.exit_survey'
    }
    
]

def left(s, amount):
    return s[:amount]
#left(str(datetime.datetime.now()), 10).replace('-','')




for file in qualtrics_files:
    automationId = file['automationId']
    file_name = file['file_name']
    print(file_name)
    
    
    current_date = left(str(datetime.datetime.now()), 10).replace('-','')
    
    baseUrl = "https://sjc1.qualtrics.com/automations-file-service/automations/{1}/files".format(dataCenter, automationId)
    
    f = tempfile.NamedTemporaryFile(delete=False)
    temp_file_path= f.name

    
    print ('Sending', file_name, temp_file_path)

    
    if file_name=='Exit_survey':
        rows = exit_survey.to_pandas()
    
        
   
    if not rows.empty:
        rows.to_csv(temp_file_path, sep='|',quotechar='"',quoting=csv.QUOTE_MINIMAL, index=False, encoding='UTF-8')
    

        with open(temp_file_path, 'r',encoding='UTF-8') as uploadfile:
            response = requests.post(baseUrl, files={ 'file': uploadfile }, headers=headers, verify=False)
            print(response.text)
            print(uploadfile)
            print('would of sent :', uploadfile)
